<a href="https://colab.research.google.com/github/sidhu2690/HMC/blob/main/Stock_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd

stock_symbol = "SUZLON.NS"
start_date = "2001-01-01"
end_date = "2023-12-31"

data = yf.download(stock_symbol, start=start_date, end=end_date, interval="1d")

data.index = pd.to_datetime(data.index)

monthly_data = data.resample('M').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})

print(monthly_data.head())


[*********************100%***********************]  1 of 1 completed

                  Open        High         Low       Close     Volume
Date                                                                 
2005-10-31  117.347748  132.222809  106.558731  131.231140  303381586
2005-11-30  130.386383  163.047256  125.758583  156.335114  173494028
2005-12-31  161.587296  174.423920  147.657990  164.296021  153215684
2006-01-31  165.278519  218.902206  144.526871  214.375412   79259386
2006-02-28  212.291061  213.521469  182.403198  201.694885   69374017


In [2]:
pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=769d963bedef0e811c50d0bc41e45eff59771a34d225679dbfa20812ed9a69e4
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
import pandas_ta as ta

monthly_data['RSI'] = ta.rsi(monthly_data['Close'], length=14)
monthly_data['Short_MA'] = ta.sma(monthly_data['Close'], length=5)
monthly_data['Long_MA'] = ta.sma(monthly_data['Close'], length=10)
macd = ta.macd(monthly_data['Close'], fast=12, slow=26, signal=9)
monthly_data['MACD'] = macd['MACD_12_26_9']
monthly_data['MACD_Signal'] = macd['MACDs_12_26_9']
bbands = ta.bbands(monthly_data['Close'], length=20, std=2)
monthly_data['Upper_BB'] = bbands['BBU_20_2.0']
monthly_data['Lower_BB'] = bbands['BBL_20_2.0']

monthly_data['Next_Month_Close'] = monthly_data['Close'].shift(-1)

monthly_data['Price_Change_%'] = (monthly_data['Next_Month_Close'] - monthly_data['Close']) / monthly_data['Close'] * 100

def generate_label(row):
    if row['Price_Change_%'] >= 10:
        return 1  # Buy
    elif row['Price_Change_%'] > 0:
        return 2  # Hold
    else:
        return 3  # Sell

monthly_data['Label'] = monthly_data.apply(generate_label, axis=1)

monthly_data.dropna(inplace=True)

final_data = monthly_data[['RSI', 'Short_MA', 'Long_MA', 'MACD', 'Upper_BB', 'Lower_BB', 'Label']]

final_data.head()


,RSI,Short_MA,Long_MA,MACD,Upper_BB,Lower_BB,Label
Date,,,,,,,
2008-07-31,44.744597,232.758057,277.228076,12.395778,354.596873,151.285766,3
2008-08-31,44.113575,224.356400,261.006909,7.549356,355.080853,146.861730,3
2008-09-30,36.812204,199.546255,240.136823,-1.164427,361.169018,133.755655,3
2008-10-31,28.512642,156.757484,208.645757,-15.866986,383.521236,96.339219,3
2008-11-30,28.336964,124.748547,184.106489,-27.406696,399.721917,65.571072,1


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

X = final_data.drop('Label', axis=1)
y = final_data['Label']

y_categorical = to_categorical(y - 1)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

y_pred_prob = model.predict(X_test_scaled)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy() + 1

y_test_labels = tf.argmax(y_test, axis=1).numpy() + 1

print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 153ms/step - accuracy: 0.3392 - loss: 1.1051 - val_accuracy: 0.1333 - val_loss: 1.1450
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.2921 - loss: 1.1118 - val_accuracy: 0.1667 - val_loss: 1.0859
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4341 - loss: 1.0601 - val_accuracy: 0.4000 - val_loss: 1.0429
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4083 - loss: 1.0667 - val_accuracy: 0.5667 - val_loss: 1.0095
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4245 - loss: 1.0473 - val_accuracy: 0.6333 - val_loss: 0.9854
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5250 - loss: 1.0098 - val_accuracy: 0.6667 - val_loss: 0.9672
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5211 - loss: 1.0282 - val_accuracy: 0.7000 - val_loss: 0.9516
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.4612 - loss: 1.0447 - val_accuracy: 0.7000 - val_loss: 0.9434

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
